In [1]:
import shapefile
import geopandas as gpd
import os
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import rasterio as rio
from rasterio.features import rasterize
import xarray as xr
import rioxarray as rxr
import requests
from skimage.measure import block_reduce
from tqdm.notebook import tqdm
from tzwhere import tzwhere
import multiprocessing as mp

In [2]:
FIRE_PERIM_DIR="../data/"
FIRE_EVENT="fired_conus-ak_events_nov2001-march2021.gpkg"
FIRE_DAY="fired_conus-ak_daily_nov2001-march2021.gpkg"

In [3]:
fire_event_df=gpd.read_file(os.path.join(FIRE_PERIM_DIR,FIRE_EVENT))

In [4]:
fire_day_df=gpd.read_file(os.path.join(FIRE_PERIM_DIR,FIRE_DAY))

KeyboardInterrupt: 

In [ ]:
#fire_event_df_rpj=fire_event_df.to_crs("EPSG:4326")

In [ ]:
fig, ax= plt.subplots(figsize=(5,5))
ax.hist(fire_event_df.area/1e6, bins=np.logspace(start=np.log10(fire_event_df.area.min()/1e6), stop=np.log10(fire_event_df.area.max()/1e6), num=25))
ax.set_xscale("log")
ax.set_xlabel("Total Burned Area ($km^2$)")
ax.set_ylabel("Frequency")
plt.show()

#### Plot the longest burning fire in the dataset

In [ ]:
plt.figure(figsize=(5,5))
longest_event_index=fire_event_df.iloc[fire_event_df.event_dur.argmax()]["id"]
longest_fire=fire_day_df[fire_day_df["id"]==387281]
date_range=pd.to_datetime(longest_fire.date, infer_datetime_format=True)
plt.plot(date_range, longest_fire.area/1e6)
plt.ylabel("Daily Burned Area($km^2$)")
plt.xticks([date_range.iloc[0],date_range.iloc[40], date_range.iloc[80]])
plt.show()

In [ ]:
longest_fire.to_crs("EPSG:4326").plot("event_day", legend=True)
plt.show()

### Select Medium Scale Fire Events

In [ ]:
medium_fire_event=fire_event_df[(fire_event_df.area >= 10e6) & (fire_event_df.area <= 30e6) & (fire_event_df.event_dur > 3)]
medium_event_bounds=medium_fire_event.bounds
medium_event_bounds["width"]=((medium_event_bounds["maxx"]-medium_event_bounds["minx"])/463.3127165283333).astype(int)
medium_event_bounds["height"]=((medium_event_bounds["maxy"]-medium_event_bounds["miny"])/463.3127165283333).astype(int)

def return_transform_object(west, south, east, north, width, height):
    return rio.transform.from_bounds(west, south, east, north, width, height)

medium_event_bounds["transform"]=medium_event_bounds.apply(lambda x: return_transform_object(x["minx"], x["miny"], x["maxx"], x["maxy"], x["width"], x["height"]), axis=1)
medium_fire_event=medium_fire_event.join(medium_event_bounds)

medium_fire_day=fire_day_df[[True if a in medium_fire_event["id"].values else False for a in fire_day_df["id"] ]]

medium_fire_day["event_id"]=medium_fire_day["id"].values
medium_fire_event["event_id"]=medium_fire_event["id"].values

medium_fire_event["date"]=pd.to_datetime(medium_fire_event["ig_date"])
medium_fire_event["envelope_centroid"]=medium_fire_event.envelope.centroid.to_crs("EPSG:4326")

### Use Global MODIS subset tool to retrieve NDVI

In [ ]:
"""
ndvi_continue=[]
for i in tqdm(range(len(medium_fire_event)-3651)):
    i=i+3651
    env_cen_x=medium_fire_event["envelope_centroid"].iloc[i].__geo_interface__["coordinates"][0]
    env_cen_y=medium_fire_event["envelope_centroid"].iloc[i].__geo_interface__["coordinates"][1]

    BASE_URL="https://modis.ornl.gov/rst/api/v1"
    MODIS_VI_SUBSET="/MOD13Q1/subset"
    MODIS_VI_DATES="/MOD13Q1/dates"
    date_r=requests.get(BASE_URL+MODIS_VI_DATES, params={"longitude": env_cen_x, "latitude": env_cen_y})

    res=pd.DataFrame(date_r.json()["dates"])
    res["date"]=pd.to_datetime(res["calendar_date"])

    date_idx=abs(res["date"]-medium_fire_event.iloc[i]["date"]).idxmin()

    while (res["date"][date_idx]+ np.timedelta64(16, "D")>=medium_fire_event.iloc[i]["date"]):
        date_idx=date_idx-1

    modis_date=res["modis_date"][date_idx]

    data_r=requests.get(BASE_URL+MODIS_VI_SUBSET, params={"longitude": env_cen_x,
                                                          "latitude": env_cen_y,
                                                          "startDate": modis_date,
                                                          "endDate": modis_date,
                                                          "kmAboveBelow":10,
                                                          "kmLeftRight":10,"band":["250m_16_days_NDVI",]})

    ndvi_data=np.array(data_r.json()["subset"][0]["data"]).reshape(81,81)

    xleft=float(data_r.json()["xllcorner"])
    ybot=float(data_r.json()["yllcorner"])
    cs=data_r.json()["cellsize"]

    ytop=ybot+ 81 * cs
    xright=xleft+81*cs

    t=rio.transform.from_bounds(xleft, ybot, xright, ytop, 81, 81)

    b=medium_fire_event.iloc[i].geometry.envelope.bounds

    bounds=rio.transform.rowcol(t, b[0], b[3]), rio.transform.rowcol(t, b[2], b[1])
    
    assert abs(rio.transform.xy(t, bounds[0][0], bounds[0][1] + 1)[0] -  cs* 0.5 - b[0]) < 0.5 * cs
    assert abs(rio.transform.xy(t, bounds[0][0], bounds[0][1])[1] -  (cs* 0.5 + b[3]))  < 0.5 * cs
    
    h=int((bounds[1][0]-(bounds[0][0]+1))/2)
    w=int((bounds[1][1]- (bounds[0][1]+1))/2)

    dim=32

    h_dif=dim-h
    w_dif=dim-w
    left_pad=w_dif//2
    right_pad=w_dif-left_pad
    top_pad=h_dif//2
    bottom_pad=h_dif-top_pad

    ndvi_data_s=ndvi_data[bounds[0][0]+1-2*top_pad: bounds[1][0]+2*bottom_pad, bounds[0][1]+1-2*left_pad:bounds[1][1]+2*right_pad]
    
    ndvi_data_reduced=block_reduce(ndvi_data_s, (2,2), np.mean)/1e4
    
    assert ndvi_data_reduced.shape[0] == 32 and ndvi_data_reduced.shape[1] == 32

    ndvi_continue.append(ndvi_data_reduced)
    """

In [ ]:
medium_fire_event["ndvi"]=np.load("../data/ndvi_all.npy").tolist()

### Get Landcover Dataset

In [ ]:
"""
lc_crop_list_continue=[]
for i in tqdm(range(len(medium_fire_event) - 4471)):
    i = i + 4471
    env_cen_x=medium_fire_event["envelope_centroid"].iloc[i].__geo_interface__["coordinates"][0]
    env_cen_y=medium_fire_event["envelope_centroid"].iloc[i].__geo_interface__["coordinates"][1]

    BASE_URL="https://modis.ornl.gov/rst/api/v1"
    MODIS_VI_SUBSET="/MCD12Q1/subset"
    MODIS_VI_DATES="/MCD12Q1/dates"
    date_r=requests.get(BASE_URL+MODIS_VI_DATES, params={"longitude": env_cen_x, "latitude": env_cen_y})

    res=pd.DataFrame(date_r.json()["dates"])
    res["date"]=pd.to_datetime(res["calendar_date"])

    date_idx=abs(res["date"]-medium_fire_event.iloc[i]["date"]).idxmin()

    while (res["date"][date_idx].year>=medium_fire_event.iloc[i]["date"].year and date_idx > 0):
        date_idx=date_idx-1

    modis_date=res["modis_date"][date_idx]

    data_r=requests.get(BASE_URL+MODIS_VI_SUBSET, params={"longitude": env_cen_x,
                                                          "latitude": env_cen_y,
                                                          "startDate": modis_date,
                                                          "endDate": modis_date,
                                                          "kmAboveBelow":10,
                                                          "kmLeftRight":10,
                                                          "band": ["LC_Type1"]}).json()

    n_rows=data_r["nrows"]
    n_cols=data_r["nrows"]
    land_cover=np.array(data_r["subset"][0]["data"]).reshape(n_rows, n_cols)
    xleft=float(data_r["xllcorner"])
    ybot=float(data_r["yllcorner"])
    cs=data_r["cellsize"]

    ytop=ybot+ n_rows * cs
    xright=xleft+n_cols*cs

    t=rio.transform.from_bounds(xleft, ybot, xright, ytop, n_rows, n_cols)

    b=medium_fire_event.iloc[i].geometry.envelope.bounds

    bounds=rio.transform.rowcol(t, b[0], b[3]), rio.transform.rowcol(t, b[2], b[1])
    ul=list(bounds[0])

    x_err=rio.transform.xy(t, ul[0], ul[1], offset="center")[0] -  (cs* 0.5 + b[0])
    y_err=rio.transform.xy(t, ul[0], ul[1], offset="center")[1] -  (b[3] - cs* 0.5)

    if x_err < -cs/2:
        ul[1]=ul[1]+1
    elif (x_err > cs/2):
        ul[1]=ul[1]-1

    if y_err > cs/2:
        ul[0]=ul[0]+1
    elif y_err < -cs/2:
        ul[0]=ul[0]-1

    h=medium_fire_event.iloc[i].height
    w=medium_fire_event.iloc[i].width

    xe=rio.transform.xy(t, ul[0], ul[1], offset="center")[0] -  (cs* 0.5 + b[0])
    ye=rio.transform.xy(t, ul[0], ul[1], offset="center")[1] -  (b[3] - cs* 0.5)

    #print(xe, ye)

    dim=32

    h_dif=dim-h
    w_dif=dim-w
    left_pad=w_dif//2
    right_pad=w_dif-left_pad
    top_pad=h_dif//2
    bottom_pad=h_dif-top_pad

    lc_crop=land_cover[ul[0]-top_pad:ul[0]+h+bottom_pad, ul[1]-left_pad:ul[1]+w+right_pad]

    lc_crop_list_continue.append(lc_crop)
"""

In [ ]:
medium_fire_event["land_cover"]=np.load("../data/lc_crop_all.npy").tolist()

### Add Digital Elevation Model

In [ ]:
dem_img=rio.open("../data/US_GMTED_MODIS.tif")
dem=dem_img.read(1)

In [ ]:
dem_list=[]
for i in range(len(medium_fire_event)):
    g=medium_fire_event.iloc[i].geometry.bounds
    ul=list(rio.transform.rowcol(dem_img.transform, g[0], g[3]))
    cell_size=463.43036
    x_err=(rio.transform.xy(dem_img.transform, ul[0], ul[1])[0]-463.43036/2)-g[0]
    y_err=(rio.transform.xy(dem_img.transform, ul[0], ul[1])[1]+463.43036/2)-g[3]
    
    if x_err < -cell_size/2:
        ul[1]=ul[1]+1
    elif (x_err > cell_size/2):
        ul[1]=ul[1]-1

    if y_err > cell_size/2:
        ul[0]=ul[0]+1
    elif y_err < -cell_size/2:
        ul[0]=ul[0]-1
   
    h=medium_fire_event.iloc[i].height
    w=medium_fire_event.iloc[i].width
    
    
    dim=32

    h_dif=dim-h
    w_dif=dim-w
    left_pad=w_dif//2
    right_pad=w_dif-left_pad
    top_pad=h_dif//2
    bottom_pad=h_dif-top_pad
    
    dem_crop=dem[ul[0]-top_pad:ul[0]+h+bottom_pad, ul[1]-left_pad:ul[1]+w+right_pad]
    
    dem_list.append(dem_crop)
    

In [ ]:
medium_fire_event["dem"]=dem_list

### Rasterize Fire Perimeters

In [ ]:
medium_fire=medium_fire_day.merge(medium_fire_event[["event_id", "minx", "miny", "maxx", "maxy", "width", "height", "transform", "dem", "ndvi", "land_cover", "envelope_centroid"]], on="event_id")
medium_fire=medium_fire[(medium_fire.width < 32) & (medium_fire.height < 32)]

def return_raster(geom, height, width, trans):
    return rasterize([geom,], out_shape=(height, width), transform=trans)
medium_fire["raster"]=medium_fire.apply(lambda x: return_raster(x["geometry"], x["height"], x["width"], x["transform"]), axis=1)

def padding(org, dim=32):
    h=org.shape[0]
    w=org.shape[1]
    h_dif=dim-h
    w_dif=dim-w
    left_pad=w_dif//2
    right_pad=w_dif-left_pad
    top_pad=h_dif//2
    bottom_pad=h_dif-top_pad
    return np.pad(org, ((top_pad, bottom_pad), (left_pad, right_pad)), 'constant', constant_values=0)

medium_fire["padded_raster"]=medium_fire["raster"].apply(padding)

In [ ]:
fig, axs=plt.subplots(5,6, figsize=(20,20))
ax=axs.flatten()
longest_fire=medium_fire[medium_fire.event_id==154860]
for i in range(30):
    ax[i].imshow(longest_fire["padded_raster"].iloc[i])
    ax[i].set_title("Fire Day {}".format(longest_fire["event_day"].iloc[i]))

### Get Meterological Variables

In [ ]:
import urllib3
import urllib
import cdsapi
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
c = cdsapi.Client(quiet=True)

In [ ]:
medium_fire["date"]=pd.to_datetime(medium_fire.date)

In [ ]:
medium_fire_no_ak=medium_fire[[medium_fire["envelope_centroid"].iloc[i].__geo_interface__["coordinates"][1] < 49.5 for i in range(len(medium_fire))]]
medium_fire_no_ak_hi=medium_fire_no_ak[[medium_fire_no_ak["envelope_centroid"].iloc[i].__geo_interface__["coordinates"][0] > -125 for i in range(len(medium_fire_no_ak))]]

In [ ]:
%%time
u10_list=[]
v10_list=[]
t2m_list=[]

for i in tqdm(range(len(medium_fire_no_ak_hi))):
    c = cdsapi.Client(quiet=True)
    env_cen_x=medium_fire_no_ak_hi["envelope_centroid"].iloc[i].__geo_interface__["coordinates"][0]
    env_cen_y=medium_fire_no_ak_hi["envelope_centroid"].iloc[i].__geo_interface__["coordinates"][1]
    hour=((np.array([2, 8, 14,  22]) - np.floor((-125+ 7.5) / 15)) % 24).tolist()
    this_day=medium_fire_no_ak_hi["date"].iloc[i]
    next_day=this_day+np.timedelta64(1, 'D')

    day_list=[]
    month_list=[]
    year_list=[]
    
    day_list.append(this_day.day)
    day_list.append(next_day.day)
    month_list.append(this_day.month)
    year_list.append(this_day.year)    
    if this_day.month != next_day.month:
        month_list.append(next_day.month)
    if this_day.year != next_day.year:
        year_list.append(next_day.year)

    day_list=[str(d).zfill(2) for d in day_list]
    month_list=[str(m).zfill(2) for m in month_list]
    year_list=[str(y) for y in year_list]

    fl=c.retrieve(
        'reanalysis-era5-land',
        {
            'format': 'netcdf',
            'variable': [
                '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
            ],
            'day': day_list,
            'year': year_list,
            'month': month_list,
            'time': [str(int(h)).zfill(2)+":00" for h in hour],
            'area': [
                env_cen_y, env_cen_x, env_cen_y-0.1,
                 env_cen_x+0.1,
            ],
        })

    with urllib.request.urlopen(fl.location) as f:
        ds = xr.open_dataset(f.read())

    time_list=[this_day + np.timedelta64(int(hour[0]), 'h'),
    this_day + np.timedelta64(int(hour[1]), 'h'),
    this_day + np.timedelta64(int(hour[2]), 'h'),
    next_day + np.timedelta64(int(hour[3]), 'h')]

    u10=ds.sel(time=time_list).u10[:,0,0].values
    v10=ds.sel(time=time_list).v10[:,0,0].values
    t2m=ds.sel(time=time_list).t2m[:,0,0].values
    
    u10_list.append(u10)
    v10_list.append(v10)
    t2m_list.append(t2m)